Задание:
Классификация текста.
Требуется написать программу, которая по отрывку текста сможет сказать,
принадлежит ли он перу Льва Толстого или Ильи Ильфа и Евгения Петрова.

Комментарий:
Оптимальной моделью для данной задачи мне представляется классификатор по метрике Term frequency - Inverse document frequency на основе SVM с линейным ядром - классическое, быстрое и эффективное решение. Так как каждый раз margin строится лишь между двумя объектами, линейного ядра вполне достаточно. Дополнительно, мы получаем возможность вывода опорных объектов (слов), что может дать лучшее понимание характера классифицируемых данных.

In [1]:
#Python v2.7
from sklearn.datasets import load_files
text_samples = load_files('Train')

from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(max_df = 0.8, ngram_range = (1,2))
                      #token_pattern = u'(?u)\\b\\w\\w\\w\\w+\\b')
tf_data = tfv.fit_transform(text_samples.data)

inv_voc = {v: k for k, v in tfv.vocabulary_.iteritems()} # Inverted vocabulary
def get_top10_tfidf(i):
    tf_indices = abs(tf_data[i].toarray()[0]).argsort()[-10:]
    top_words = [inv_voc[index] for index in tf_indices]
    top_words.sort()
    print(','.join(top_words))
    print(text_samples.target_names[text_samples.target[i]])

from sklearn.svm import LinearSVC
clf = LinearSVC() # Because 2 classes
from sklearn.model_selection import KFold


Оптимизация параметра С по GridSearch оказалась бесполезна

In [7]:
'''from sklearn.model_selection import GridSearchCV
import numpy as np
grid = {'C': np.power(10.0, np.arange(-5, 6))}
gs = GridSearchCV(clf, grid, scoring='accuracy',
                  cv=KFold(n_splits = 5, shuffle = True))
gs.fit(tf_data, text_samples.target)

for a in gs.grid_scores_:
	print a.mean_validation_score
	print a.parameters'''

"from sklearn.model_selection import GridSearchCV\nimport numpy as np\ngrid = {'C': np.power(10.0, np.arange(-5, 6))}\ngs = GridSearchCV(clf, grid, scoring='accuracy',\n                  cv=KFold(n_splits = 5, shuffle = True))\ngs.fit(tf_data, text_samples.target)\n\nfor a in gs.grid_scores_:\n\tprint a.mean_validation_score\n\tprint a.parameters"

In [10]:
# C parameter optimization gave no results
clf.fit(tf_data, text_samples.target)

# Load test samples
test_samples = load_files('Test')
test_data = tfv.transform(test_samples.data)
test_targets = clf.predict(test_data)
print 'Predicted: ',test_targets, '\nActual:    ', test_samples.target
print 'Accuracy: ',\
    1 - sum(abs(test_targets - test_samples.target))/float(len(test_targets))
'''for i in xrange(len(test_targets)):
	if test_targets[i] != test_samples.target[i]:
		print i, '\n', test_samples.data[i], '\n'   '''
indices_1 = clf.coef_[0].argsort()[-10:]
print 'Опорные слова'
print 'Толстой: ', ', '.join([inv_voc[index] for index in indices_1])
indices_0 = clf.coef_[0].argsort()[:10]
print 'Ильф и Петров: ', ', '.join([inv_voc[index] for index in indices_0])

Predicted:  [0 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0] 
Actual:     [0 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0]
Accuracy:  1.0
Опорные слова
Толстой:  сь, мне, то что, de, всё, анна, меня, хаджи, она, жизни
Ильф и Петров:  ипполит матвеевич, матвеевич, ипполит, остап, федор, бабский, отец федор, отец, гейнрих, нью


Кросс-валидация бесполезна для такого типа задач - слишком важна обучающая выборка и её целостность в случае небольшого датасета